# Wetterskip Fryslan
### Onderscheid water van niet-water obv RGB image en Class image

#### Inlezen validatiefoto 1 met bijbehorende class afbeelding

#### Inlezen pickle file van opgeslagen modellen:
    - Naive Bayes
    - Logistic Regression
    - RandomForrest

#### Bepalen uitkomst modellen op validatie foto's.

## Bibliotheken laden

In [1]:
import numpy as np
import pickle
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB

## Open Class van 1e validatie luchtfoto

In [2]:
# open class bestand in ZWART-WIT (convert('l'))
ClassImgVal1 = Image.open("validatie1-class.tif").convert('L')
# zet om naar array (matrix)
ClassArrayVal1 = np.array(ClassImgVal1)
# zet breedte en hoogte om in variabelen
ClassImageWidth, ClassImageHeight = ClassArrayVal1.shape

In [3]:
# vervang alle waarden van 255 (wit) door 1 en alle andere door 0 (zwart)
ClassArrayVal1[ClassArrayVal1 < 255] = 1 # zwart/water
ClassArrayVal1[ClassArrayVal1 == 255] = 0 # wit/geen water

In [4]:
# grootte van het sliding window
WindowLength = 5
WindowSize = WindowLength**2
# variable ter bepaling van de hoeveelheid pixels naar rechts en naar beneden (de 'slide')
Step = 1 
# tijdelijke teller
Counter = 0 
# drempel waarboven een window als water wordt geclassificeerd
Threshold = .8 

# bepaal hoe groot de output vector wordt en creeer een lege array:
ClassArrayResultVal1 = np.zeros(shape=(((ClassImageWidth - WindowLength) * (ClassImageHeight - WindowLength))))

# voor elke rij
for i in range(0,ClassImageWidth-WindowLength,Step):
    # voor elke kolom
    for j in range(0, ClassImageHeight-WindowLength,Step):
        # wanneer de som van de waarden in het window gelijk is aan het totale aantal pixels in het window dan
        # zitten er dus alleen maar 1'en in --> dus water
        if (ClassArrayVal1[i:i + WindowLength, j:j + WindowLength]==1).sum() == (WindowSize):
            ClassArrayResultVal1[Counter] = 1
        # wanneer de som van de waarden in het window gelijk is aan nul dan zitten er dus geen 1'en in --> dus geen water
        elif (ClassArrayVal1[i:i + WindowLength, j:j + WindowLength]==1).sum() == 0:
            ClassArrayResultVal1[Counter] = 0
        # wanneer het aantal 1'en gedeeld door het aantal pixels groter is dan de toegestane drempel, dan alsnog 
        # classificeren als 'water'
        elif ((ClassArrayVal1[i:i + WindowLength, j:j + WindowLength]==1).sum() / (WindowSize)) > Threshold:
            ClassArrayResultVal1[Counter] = 1
        # in de resterende gevallen is het onbekend
        else:
            ClassArrayResultVal1[Counter] = 0
            
        Counter += 1

In [5]:
ClassArrayResultVal1.shape

(990025,)

## Openen Feature Validatiefoto 1 (RGB)

In [6]:
# open feature foto (RGB).
FeatureImgVal1 = Image.open("validatie1.png").convert('RGB')
# zet om naar array (matrix)
FeatureArrayVal1 = np.array(FeatureImgVal1)
# zet breedte, hoogte en diepte om in variabelen
FeatureImageWidth, FeatureImageHeight, FeatureImageLayers = FeatureArrayVal1.shape

In [7]:
# bepaal het formaat van de output matrix en creeer een lege array:
FeatureArrayResultVal1 = np.zeros(shape=(((FeatureImageWidth - WindowLength) * (FeatureImageHeight - WindowLength)), (WindowSize)*FeatureImageLayers))

# voor elke layer
for z in range(0, FeatureImageLayers):
    Counter = 0
    # voor elke rij
    for i in range(0, FeatureImageWidth-WindowLength, Step):
        # voor elke kolom
        for j in range(0, FeatureImageHeight-WindowLength, Step):
            # zet de betreffende waarden per layer in feature matrix
            FeatureArrayResultVal1[Counter, (z*(WindowSize)):(z*(WindowSize)+(WindowSize))] = (FeatureArrayVal1[i:i + WindowLength, j:j + WindowLength, z]).flatten()          
            Counter += 1


In [8]:
X_val1 = FeatureArrayResultVal1
y_val1 = ClassArrayResultVal1

## Naive Bayes (baseline)

### Inlezen model met Naive Bayes

In [10]:
# open het getrainde Naive Bayes model van schijf
filename = 'GaussianNaiveBayes_model.sav'
gnb_clf = pickle.load(open(filename, 'rb'))

### Voorspel met Gaussian Naive Bayes

In [11]:
# bekijk resultaten
gnb_predictions = gnb_clf.predict(X_val1)
print(classification_report(y_val1, gnb_predictions))

             precision    recall  f1-score   support

        0.0       0.92      0.70      0.79    584259
        1.0       0.68      0.91      0.78    405766

avg / total       0.82      0.78      0.79    990025



## Logistic Regression

### Inlezen Logistic Regression model

In [12]:
# open het getrainde LogisticRegression model van schijf
filename = 'Logistic_Regression_model.sav'
classifier = pickle.load(open(filename, 'rb'))

### Voorspelling Logistic Regression model

In [13]:
# bekijk resultaten
predictions = classifier.predict(X_val1)
print(classification_report(y_val1, predictions))

             precision    recall  f1-score   support

        0.0       0.99      0.96      0.97    584259
        1.0       0.94      0.98      0.96    405766

avg / total       0.97      0.97      0.97    990025



## Random Forrest

### Inlezen Random Forrest model

In [14]:
filename = 'RandomForrestClassifier_model.sav'
rf_clf = pickle.load(open(filename, 'rb'))

### Voorspel uitkomt op basis van Random Forrest

In [15]:
rf_pred = rf_clf.predict(X_val1)
print(classification_report(y_val1, rf_pred))

             precision    recall  f1-score   support

        0.0       0.96      0.98      0.97    584259
        1.0       0.98      0.95      0.96    405766

avg / total       0.97      0.97      0.97    990025



### Opslaan prediction met Pickle

In [16]:
filename = 'RandomForrestClassifier_Prediction_Validation1.sav'
pickle.dump(rf_pred, open(filename, 'wb'))